In [27]:
import mbuild as mb
import itertools
import math

import gsd.hoomd
import hoomd
import numpy
import foyer

In [28]:
class btoCompound(mb.Compound):
    def __init__(self):
        super(btoCompound, self).__init__()
        particle_A1 = mb.Particle(name="_A1", pos=[0, 0, 0])
        particle_D1 = mb.Particle(name="_D", pos=[0.6, -0.4, 0])
        particle_D2 = mb.Particle(name="_D", pos=[-0.6, -0.4, 0])
        particle_A21 = mb.Particle(name="_A2", pos=[1, -1.2, 0])
        particle_A22 = mb.Particle(name="_A2", pos=[-1, -1.2, 0])
        particle_ss1 = mb.Particle(name="_SS", pos=[-0.1568, -0.6363, 0.4135])
        particle_ss2 = mb.Particle(name="_SS", pos=[0.1568, -0.6363, 0.4135])
        particle_sl11 = mb.Particle(name="_SL1", pos=[1.1, 0.02, 0])
        particle_sl12 = mb.Particle(name="_SL1", pos=[-1.1, 0.02, 0])
        particle_sl21 = mb.Particle(name="_SL2", pos=[1.2, 0.6, 0])
        particle_sl22 = mb.Particle(name="_SL2", pos=[-1.2, 0.6, 0])
        self.add(particle_A1)
        self.add(particle_D1)
        self.add(particle_D2)
        self.add(particle_A21)
        self.add(particle_A22)
        self.add(particle_ss1)
        self.add(particle_ss2)
        self.add(particle_sl11)
        self.add(particle_sl12)
        self.add(particle_sl21)
        self.add(particle_sl22)
        self.add_bond((self[0], self[1]))
        self.add_bond((self[0], self[2]))
        self.add_bond((self[1], self[3]))
        self.add_bond((self[2], self[4]))
        self.add_bond((self[1], self[5]))
        self.add_bond((self[2], self[6]))
        self.add_bond((self[1], self[7]))
        self.add_bond((self[2], self[8]))
        self.add_bond((self[7], self[9]))
        self.add_bond((self[8], self[10]))
        

bto_compound = btoCompound()   
box_bto = mb.fill_box(compound=bto_compound, n_compounds=5, box=[100, 100, 100])

/home/gwenwhite/miniconda3/envs/grits/lib/python3.8/site-packages/mbuild/conversion.py:1763: UserWarning: No element attribute associated with '<_A1 pos=([0. 0. 0.]), 2 bonds, id: 140373029236160>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/home/gwenwhite/miniconda3/envs/grits/lib/python3.8/site-packages/mbuild/conversion.py:1763: UserWarning: No element attribute associated with '<_D pos=([ 0.6 -0.4  0. ]), 4 bonds, id: 140373029236544>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/home/gwenwhite/miniconda3/envs/grits/lib/python3.8/site-packages/mbuild/conversion.py:1763: UserWarning: No element attribute associated with '<_A2 pos=([ 1.  -1.2  0. ]), 1 bonds, id: 140373029319104>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/home/gwenwhite/miniconda3/envs/grits/lib/python3.8/site-packages

In [29]:
foyer_forcefield = foyer.Forcefield("bto-ff.xml")

In [30]:
typed_bto = foyer_forcefield.apply(box_bto, verbose=False)

/home/gwenwhite/miniconda3/envs/grits/lib/python3.8/site-packages/mbuild/conversion.py:1763: UserWarning: No element attribute associated with '<_A1 pos=([60.5712 42.03   56.8524]), 2 bonds, id: 140373029322704>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/home/gwenwhite/miniconda3/envs/grits/lib/python3.8/site-packages/mbuild/conversion.py:1763: UserWarning: No element attribute associated with '<_D pos=([60.5637 41.8989 57.5615]), 4 bonds, id: 140373029322656>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/home/gwenwhite/miniconda3/envs/grits/lib/python3.8/site-packages/mbuild/conversion.py:1763: UserWarning: No element attribute associated with '<_A2 pos=([60.1127 41.7325 58.3157]), 1 bonds, id: 140373029321888>'; and no matching elements found based upon the compound name. Setting atomic number to zero.
  warn(warn_msg)
/home/gwenwhite/miniconda3/envs/

In [31]:
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

In [32]:
snapshot, forcefield, refs = create_hoomd_forcefield(
    typed_bto,
    auto_scale=True,
    r_cut=2.5
)

Processing LJ and QQ
No charged groups found, ignoring electrostatics
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions


In [33]:
gpu = hoomd.device.GPU()
sim = hoomd.Simulation(device=gpu, seed=1,)

In [34]:
sim.create_state_from_snapshot(snapshot)

## Build Integrator

In [35]:
bonds_k = 3.
angle_k = 3.
dihedral_k = 3.

epsilon = 1
sigma = 1
r_cut = 2.5
KT = 1.5

In [36]:
integrator = hoomd.md.Integrator(dt=0.005)
integrator.forces.extend(forcefield)
# NVT
nvt = hoomd.md.methods.NVT(kT=KT, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

## Box Compressor

In [37]:
rho = sim.state.N_particles / sim.state.box.volume
rho

2.161089681640111e-06

In [38]:
initial_box = sim.state.box
final_box = hoomd.Box.from_box(initial_box)  # make a copy of initial_box
final_rho = 0.1
final_box.volume = sim.state.N_particles / final_rho

In [39]:
initial_box

hoomd.box.Box(Lx=294.1462390747833, Ly=294.1462390747833, Lz=294.1462390747833, xy=0.0, xz=0.0, yz=0.0)

In [40]:
final_box

hoomd.box.Box(Lx=8.19321270600646, Ly=8.19321270600646, Lz=8.19321270600646, xy=0.0, xz=0.0, yz=0.0)

In [41]:
ramp = hoomd.variant.Ramp(A=0, B=1, t_start=sim.timestep, t_ramp=20000)

In [42]:
box_resizer = hoomd.update.BoxResize(trigger=hoomd.trigger.Periodic(100),
                                    box1=initial_box,
                                    box2=final_box,
                                    variant=ramp)

In [43]:
sim.operations.updaters.append(box_resizer)

In [44]:
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=KT)
thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
    filter=hoomd.filter.All())
sim.operations.computes.append(thermodynamic_properties)
sim.run(0)

In [45]:
logger = hoomd.logging.Logger()
logger.add(thermodynamic_properties, quantities=["potential_energy","kinetic_energy", "kinetic_temperature", "pressure"])
gsd_writer = hoomd.write.GSD(filename='cg_bto_traj.gsd',
                             trigger=hoomd.trigger.Periodic(1000),
                             mode='wb',
                             filter=hoomd.filter.All())
sim.operations.writers.append(gsd_writer)
gsd_writer.log = logger

In [46]:
sim.run(1e5)